## Mounter

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Function Declaration

In [0]:
import tensorflow as tf
import numpy as np
import os
import nibabel as nib
import cv2 as cv
import matplotlib.pyplot as plt
    
def modality(Path,index):
    X = []
    X_per_paitent = []
    p=os.listdir(Path) 
    recs_in=[]
    counter=0
    counter_2=0

    for i in p[:14]:                                                                      # Loading all the folders in the given path
        q = os.listdir(os.path.join(Path,i))     

        x = nib.load(os.path.join(Path,i,q[index]))         
        f = x.get_fdata()
        f = np.asarray(f,'float32')
        
        ct=0
        recs_in.append(f.shape[2])
        #print(counter_2)
        counter_2+=1
        for j in range(f.shape[2]):                                                        # Processing the MRI Scan in the axial view
            _slice = cv.resize(f[:,:,j],(256,256),interpolation=cv.INTER_NEAREST)             # Resizing the slice to the shape(256,256)
            if(index not in [3,4,5,6,7,8,9] and np.sum(_slice) != 0 ):  
                if index==1:
                  ct+=1 
                  counter+=1                                         # To check whether the slice is null or not
              #  _slice = _slice / (np.max(_slice) + 0.00001)                               # Normalization
                  _slice = (_slice - np.mean(_slice) + 0.00001) / (np.std(_slice) + 0.00001)
                   # Standardization
                else:
                                              # To check whether the slice is null or not
              #  _slice = _slice / (np.max(_slice) + 0.00001)                               # Normalization
                  _slice = (_slice - np.mean(_slice) + 0.00001) / (np.std(_slice) + 0.00001) # Standardization
            elif(index in [3,4,5,6,7,8,9]):   # if index = 3, Then it is output mask and we don't normalize or standardize it 
                _slice = np.array(_slice)
                _slice[_slice > 0] = 1.0
                _slice[_slice < 0] = 0.0
            _slice = _slice.T
            _slice = _slice[:,:,np.newaxis]
            X.append(_slice)
   # X=np.array(X,dtype='float32')
    return X,recs_in

In [0]:
def remove_null_samples(X_Dp, X_Flair, X_Gado, X_T1, X_T2, Y_Manual,recs): 
     
    X=[]
    Y=[]
    counter=0
    counter_2=0
    mult=0;
    count=0
    rec=[]
    keep_count=[]
    keep=[]
    print(recs)
    r=np.array(recs,dtype='float32')
    print(np.sum(r))
    print(len(X_Dp))

    for i in range(len(X_Dp)):  
        if counter==(recs[mult]):
          print(counter)
          mult+=1
          rec.append(count)
          counter=0
          print(counter_2)
          count=0
        final_slice = np.concatenate((X_Dp[i],X_Flair[i],X_Gado[i],X_T1[i],X_T2[i]), axis = -1)
        if(np.sum(final_slice) != 0):        # checking whether the final slice is empty or not             
            X.append(final_slice)
            Y.append(Y_Manual[i])
            
            count+=1
        counter+=1
        counter_2+=1

    
    rec.append(count)
#   Converting the list into array  
    X=np.array(X,dtype='float32')
    Y=np.array(Y,dtype='float32')
    rec=np.array(rec,dtype='float32')
    
    return X,Y,rec


In [0]:
def store_data(X,Y,rec):
    np.save("drive/My Drive/MS_data/X_Major_new.npy",X)
    np.save("drive/My Drive/MS_data/Y_Major_new.npy",Y)
    np.save("drive/My Drive/MS_data/Major_new_rec_after_removal.npy",rec)

In [0]:
import math
def Major(Y_1,Y_2,Y_3,Y_4,Y_5,Y_6,Y_7):
  Y=[]
  sum2=[]
  flag=0
  #y=np.array()
  print("A")
  for i in range (len(Y_1)):
    #print(Y_1[i])
    
        f=np.concatenate((Y_1[i],Y_2[i],Y_3[i],Y_4[i],Y_5[i],Y_6[i],Y_7[i]),axis=-1)
        sum=np.sum(f,axis=2)
       # print(sum)
        
          #print(j)
        sum_1=np.divide(sum,4)
        sum_1=np.floor(sum_1)
        sum2.append(sum_1)

    #sum2=np.array(sum2,dtype='float32')
    
        
  return sum2

## Data Preprocessing

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

#import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input ,BatchNormalization , Activation 
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


#import dataPrepare as process
# Loading all the 5 different modalities of each MRI Scan of all 15 different patients and 1st rater Manual SegmentationX_Dp      =   modality(Path,0)
#import Modified_UNet 
#import plots
#import Metrics

# Setting the path
Path='drive/My Drive/Pre-processed'



# Loading all the 5 different modalities of each MRI Scan of all 15 different patients and 1st rater Manual Segmentation
X_Dp_t,rec      =   modality(Path,0)
X_Flair_t,rec_1   =   modality(Path,1)
X_Gado_t,rec    =   modality(Path,2)
X_T1_t,rec      =   modality(Path,10)
X_T2_t,rec      =   modality(Path,11)
rec=np.array(rec_1,dtype='float32')
np.save("drive/My Drive/MS_data/Major_new_rec_before_removal.npy",rec)

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

#import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input ,BatchNormalization , Activation 
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


#import dataPrepare as process
# Loading all the 5 different modalities of each MRI Scan of all 15 different patients and 1st rater Manual SegmentationX_Dp      =   modality(Path,0)
#import Modified_UNet 
#import plots
#import Metrics

# Setting the path
Path='drive/My Drive/Pre-processed'
Y_1,rec  =   modality(Path,3)
Y_2,rec  =   modality(Path,4)
Y_3,rec  =   modality(Path,5)
Y_4,rec  =   modality(Path,6)
Y_5,rec  =   modality(Path,7)
Y_6,rec  =   modality(Path,8)
Y_7,rec  =   modality(Path,9)

In [0]:
Y_Manual=Major(Y_1,Y_2,Y_3,Y_4,Y_5,Y_6,Y_7)

A


In [0]:
import numpy as np

np.save("drive/My Drive/MS_data/Y_manual_new.npy",Y_Manual)

In [0]:
Y_manual=list(Y_Manual)

In [0]:
Y_Manual=np.load("drive/My Drive/MS_data/Y_manual_new.npy")

In [0]:
X, Y,rec = remove_null_samples(X_Dp_t, X_Flair_t, X_Gado_t, X_T1_t, X_T2_t, Y_manual,rec_1)

[256, 256, 512, 512, 256, 512, 512, 256, 512, 256, 336, 336, 336, 336]
5184.0
5184
256
256
256
512
512
1024
512
1536
256
1792
512
2304
512
2816
256
3072
512
3584
256
3840
336
4176
336
4512
336
4848


In [0]:
store_data(X,Y,rec)

In [0]:
import numpy as np
Y=np.load("drive/My Drive/MS_data/Y_Major_new.npy")

In [0]:
Y_Manual=list(Y)

In [0]:
Y_1=Y_Manual[:1401]
Y_2=Y_Manual[1401:]

In [0]:
p=[]
y=[]
z=[]
a=[]
n=1
for i in Y_1:
  print(n)
  for j in i:
    for k in j:
      #print(k)
      a.append(k)
     # print(a)
      z.append(a)
      a=[]
    #print(z)
    y.append(z)
    z=[]
  p.append(y)
  y=[]
  n+=1


#print(p)


In [0]:
temp=np.array(p)
np.save("drive/My Drive/MS_data/temp_1_new.npy",temp)

In [0]:
q=[]
y=[]
z=[]
a=[]
n=1401
for i in Y_2:
  print(n)
  for j in i:
    for k in j:
      #print(k)
      a.append(k)
     # print(a)
      z.append(a)
      a=[]
    #print(z)
    y.append(z)
    z=[]
  q.append(y)
  y=[]
  n+=1


1401
1402
1403
1404
1405
1406
1407
1408
1409
1410
1411
1412
1413
1414
1415
1416
1417
1418
1419
1420
1421
1422
1423
1424
1425
1426
1427
1428
1429
1430
1431
1432
1433
1434
1435
1436
1437
1438
1439
1440
1441
1442
1443
1444
1445
1446
1447
1448
1449
1450
1451
1452
1453
1454
1455
1456
1457
1458
1459
1460
1461
1462
1463
1464
1465
1466
1467
1468
1469
1470
1471
1472
1473
1474
1475
1476
1477
1478
1479
1480
1481
1482
1483
1484
1485
1486
1487
1488
1489
1490
1491
1492
1493
1494
1495
1496
1497
1498
1499
1500
1501
1502
1503
1504
1505
1506
1507
1508
1509
1510
1511
1512
1513
1514
1515
1516
1517
1518
1519
1520
1521
1522
1523
1524
1525
1526
1527
1528
1529
1530
1531
1532
1533
1534
1535
1536
1537
1538
1539
1540
1541
1542
1543
1544
1545
1546
1547
1548
1549
1550
1551
1552
1553
1554
1555
1556
1557
1558
1559
1560
1561
1562
1563
1564
1565
1566
1567
1568
1569
1570
1571
1572
1573
1574
1575
1576
1577
1578
1579
1580
1581
1582
1583
1584
1585
1586
1587
1588
1589
1590
1591
1592
1593
1594
1595
1596
1597
1598
1599
1600


In [0]:
temp2=np.array(q)
np.save("drive/My Drive/MS_data/temp_2_new.npy",temp2)

In [0]:
temp=np.load("drive/My Drive/MS_data/temp_1_new.npy")
p=list(temp)

In [0]:
temp2=np.load("drive/My Drive/MS_data/temp_2_new.npy")
q=list(temp2)

In [0]:
j=1401
for i in q:
  print(j)
  p.append(i)
  j+=1

In [0]:
np.save("drive/My Drive/MS_data/Y_Manual_2_new.npy",p)

In [0]:
import numpy as np

P=np.load("drive/My Drive/MS_data/Y_Manual_2_new.npy")

In [0]:
Y=p

In [0]:
Y=list(Y)

In [0]:
dummy_y=[]
d_temp_1=[]
d_temp_2=[]
for i in range(len(Y[0])):
  d_temp_1=[]
  for j in range(len(Y[0][0])):
    d_temp_2=[]
    for k in range(len(Y[0][0][0])):
      d_temp_2.append(0.)
    d_temp_1.append(d_temp_2)
  dummy_y.append(d_temp_1)
Y_dummy=np.array(dummy_y, dtype='float32')



 

In [0]:
X=np.load("drive/My Drive/MS_data/X_Major_new.npy")

In [0]:
X=list(X)

In [0]:
dummy_x=[]
d_temp_1=[]
d_temp_2=[]
for i in range(len(X[0])):
  d_temp_1=[]
  for j in range(len(X[0][0])):
    d_temp_2=[]
    for k in range(len(X[0][0][0])):
      d_temp_2.append(0.)
    d_temp_1.append(d_temp_2)
  dummy_x.append(d_temp_1)
X_dummy=np.array(dummy_x, dtype='float32')

In [0]:
import numpy as np

#Y=np.load("drive/My Drive/MS_data/Y_Manual_2_new.npy")
X=np.load("drive/My Drive/MS_data/X_Major_new.npy")

In [0]:
X=list(X)

In [0]:
extra=[0,1,6,4,6,0,5,0,4,3,1,3,3,0]

In [0]:
ext=np.array(extra,dtype='float32')

In [0]:
r=np.load("drive/My Drive/MS_data/Major_new_rec_after_removal.npy")

In [0]:
rec=list(r)

In [0]:
j=0
index=0
for i in extra:
  k=0
  
  print(i)
  index=index+rec[j]
  j+=1
  while(k<i):
    index=index+k
    print(index)
    index=int(index)
    Y.insert(index,Y_dummy)
    X.insert(index,X_dummy)

    k+=1

0
1
279.0
6
561.0
562
564
567
571
576
4
844.0
845
847
850
6
988.0
989
991
994
998
1003
0
5
1590.0
1591
1593
1596
1600
0
4
2036.0
2037
2039
2042
3
2183.0
2184
2186
1
2377.0
3
2598.0
2599
2601
3
2782.0
2783
2785
0


In [0]:
rec=list(r)
X=list(X)
Y=list(Y)

In [0]:
X=np.array(X,dtype='float32')
Y=np.array(Y,dtype='float32')


In [0]:
np.save("drive/My Drive/MS_data/Actual_X.npy",X)
np.save("drive/My Drive/MS_data/Actual_Y.npy",Y)

In [0]:
import numpy as np

Y=np.load("drive/My Drive/MS_data/Actual_X.npy")
X=np.load("drive/My Drive/MS_data/Actual_Y.npy")

In [0]:
X_1=[]
Y_1=[]

In [0]:
X=list(X)
Y=list(Y)

In [0]:
index_x_p=0
length=8
index_x=length
index_y_p=0
index_y=length
x_temp=[]
y_temp=[]



In [0]:
while index_x<=len(X) and index_y<=len(Y):
  x_temp=X[index_x_p:index_x]
  y_temp=Y[index_y_p:index_y]
  x_temp_1=np.array(x_temp,dtype='float32')
  y_temp_1=np.array(y_temp,dtype='float32')
  X_1.append(x_temp_1)
  Y_1.append(y_temp_1)
  index_y_p=index_y_p+length
  index_x_p=index_x_p+length
  index_x=index_x+length
  index_y=index_y+length


In [0]:
X_1=np.array(X_1,dtype='float32')
Y_1=np.array(Y_1,dtype='float32')


In [0]:
np.save("drive/My Drive/MS_data/train_X.npy",X_1)
np.save("drive/My Drive/MS_data/train_Y.npy",Y_1)

In [0]:
import numpy as np

X_1=np.load("drive/My Drive/MS_data/train_X.npy")
Y_1=np.load("drive/My Drive/MS_data/train_Y.npy")

## creating train test and validation sets

In [0]:
extra=[0,1,6,4,6,0,5,0,4,3,1,3,3,0]

In [0]:
r=np.load("drive/My Drive/MS_data/Major_new_rec_after_removal.npy")

In [0]:
rec=list(r)

In [0]:
j=0
final=[]
while j<len(extra):
  leng=rec[j]+extra[j]
  final.append(leng/8)
  j+=1

np.save("drive/My Drive/MS_data/Major_final_size.npy",final)

In [0]:
r=np.load("drive/My Drive/MS_data/Major_final_size.npy")

In [0]:
rec=list(r)

In [0]:
import json

i=0
file_w="drive/My Drive/MS_data/per_paitent_records.json"
prev=0
total=0
dict_k={}
while i<len(r):
  current=r[i]
  j=i+1
  dict_k[str(j)]={}
  dict_k[str(j)]["Starting"]=prev
  dict_k[str(j)]["Ending"]=total+current-1
  total+=current
  prev=total
  i+=1

with open(file_w, "w") as outfile:
			json.dump(dict_k, outfile) 


In [0]:
import random
import json
dict_l={}
list_val=[]
list_train=[]
file_w="drive/My Drive/MS_data/set_divisions.json"
list_test=[]
for i in range(5):
  patient_list=['1','2','3','4','5','6','7','8','9','10','11','12','13','14']
  for j in range(7):
    flag=0
    while(flag!=1):
      num=random.randint(1,14)
      if patient_list.count(str(num))> 0:
        list_train.append(str(num))
        patient_list.remove(str(num))
        flag=1
  if dict_l.get("train") is None:
    dict_l["train"]=[list_train]
  else:
    dict_l["train"].append(list_train)
  list_train=[]
  for j in range(3):
    flag=0
    while(flag!=1):
      num=random.randint(1,14)
      if patient_list.count(str(num))> 0:
        list_val.append(str(num))
        patient_list.remove(str(num))
        flag=1
  if dict_l.get("validate") is None:
    dict_l["validate"]=[list_val]
  else:
    dict_l["validate"].append(list_val)
  list_val=[]
  for j in range(4):
    flag=0
    while(flag!=1):
      num=random.randint(1,14)
      if patient_list.count(str(num))> 0:
        list_test.append(str(num))
        patient_list.remove(str(num))
        flag=1
  if dict_l.get("test") is None:
    dict_l["test"]=[list_test]
  else:
    dict_l["test"].append(list_test)
  list_test=[]    

with open(file_w, "w") as outfile:
			json.dump(dict_l, outfile) 

    

In [0]:
import pandas as pd
import numpy as np
import json
import os

Y=np.load("drive/My Drive/MS_data/train_Y.npy")
print(len(Y))
X=np.load("drive/My Drive/MS_data/train_X.npy")
print(len(X))
f1=open("drive/My Drive/MS_data/per_paitent_records.json")
data_1=json.load(f1)
f2=open("drive/My Drive/MS_data/set_divisions.json")
data_2=json.load(f2)
i=1
while i<=5:
  file_1="drive/My Drive/MS_data/"+str(i)
  print(file_1)
  os.mkdir(file_1)
  train_list=(data_2['train'])[i-1]
  file_1_ext=file_1+'/train'
  print(file_1_ext)
  os.mkdir(file_1_ext)
  X_temp=[]
  Y_temp=[]
  for j in train_list:
    st_index=data_1[str(j)]["Starting"]
    end_index=data_1[str(j)]["Ending"]
    X_temp.append(X[int(st_index):int(end_index)])
    Y_temp.append(Y[int(st_index):int(end_index)])
  file_1_X=file_1_ext+'/'+'X.npy'
  file_1_Y=file_1_ext+'/'+'Y.npy'
  X_temp_np=np.concatenate((X_temp[0],X_temp[1],X_temp[2],X_temp[3],X_temp[4],X_temp[5],X_temp[6]),axis=0)
  Y_temp_np=np.concatenate((Y_temp[0],Y_temp[1],Y_temp[2],Y_temp[3],Y_temp[4],Y_temp[5],Y_temp[6]),axis=0)
  print(len(X_temp_np))
  print(len(Y_temp_np))
  np.save(file_1_X,X_temp_np)
  np.save(file_1_Y,Y_temp_np)
  train_list=(data_2['validate'])[i-1]
  file_1_ext=file_1+'/validate'
  os.mkdir(file_1_ext)
  X_temp=[]
  Y_temp=[]
  for j in train_list:
    st_index=data_1[str(j)]["Starting"]
    end_index=data_1[str(j)]["Ending"]
    X_temp.append(X[int(st_index):int(end_index)])
    Y_temp.append(Y[int(st_index):int(end_index)])
  file_1_X=file_1_ext+'/'+'X.npy'
  file_1_Y=file_1_ext+'/'+'Y.npy'
  X_temp_np=np.concatenate((X_temp[0],X_temp[1],X_temp[2]),axis=0)
  Y_temp_np=np.concatenate((Y_temp[0],Y_temp[1],Y_temp[2]),axis=0)
  print(len(X_temp_np))
  print(len(Y_temp_np))
  np.save(file_1_X,X_temp_np)
  np.save(file_1_Y,Y_temp_np)
  train_list=(data_2['test'])[i-1]
  file_1_ext=file_1+'/test'
  os.mkdir(file_1_ext)
  X_temp=[]
  Y_temp=[]
  for j in train_list:
    st_index=data_1[str(j)]["Starting"]
    end_index=data_1[str(j)]["Ending"]
    X_temp.append(X[int(st_index):int(end_index)])
    Y_temp.append(Y[int(st_index):int(end_index)])
  file_1_X=file_1_ext+'/'+'X.npy'
  file_1_Y=file_1_ext+'/'+'Y.npy'
  X_temp_np=np.concatenate((X_temp[0],X_temp[1],X_temp[2],X_temp[3]),axis=0)
  Y_temp_np=np.concatenate((Y_temp[0],Y_temp[1],Y_temp[2],Y_temp[3]),axis=0)
  print(len(X_temp_np))
  print(len(Y_temp_np))
  np.save(file_1_X,X_temp_np)
  np.save(file_1_Y,Y_temp_np)
  i+=1



372
372
drive/My Drive/MS_data/1
drive/My Drive/MS_data/1/train
144
144
107
107
107
107
drive/My Drive/MS_data/2
drive/My Drive/MS_data/2/train
196
196
60
60
102
102
drive/My Drive/MS_data/3
drive/My Drive/MS_data/3/train
178
178
79
79
101
101
drive/My Drive/MS_data/4
drive/My Drive/MS_data/4/train
173
173
84
84
101
101
drive/My Drive/MS_data/5
drive/My Drive/MS_data/5/train
188
188
60
60
110
110
